In [8]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_ALL, '') 
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

import datetime
from datetime import date


# ax.legend(bbox_to_anchor=(1.04,0.5), loc="center left")

saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [9]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

In [10]:
latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir

df = pd.read_csv(latestdir+'/Regionalt_DB/03_bekraeftede_tilfaelde_doede_indlagte_pr_dag_pr_koen.csv',encoding='latin1',delimiter = ';')
df['Prøvetagningsdato'] = pd.to_datetime(df['Prøvetagningsdato'])

kortdf  = pd.read_csv(latestdir+'/Regionalt_DB/13_regionale_kort.csv',encoding='latin1',delimiter = ';')
# kortdf

In [11]:
latestsubdir

'SSI_dashboard_2022-02-08'

In [12]:


# Data is (only) in the file from the most recent tuesday. 
# Should be made smarter, but here hardcoded
# tuePath = 'SSI_data_2022-01-25'
# Now automatic finding of latest tuesday:
for k in range(0,7):
    dayToCheck = np.datetime64('today')-np.timedelta64(k,'D')
    thisWeekDay = (dayToCheck).astype(datetime.datetime).isoweekday()    
    if (thisWeekDay == 2):
        tuePath = 'SSI_data_'+str(dayToCheck)

print(f'Path to latest Tuesday was {tuePath}')

dirPath = path_data + tuePath + '\\'

df1 = pd.read_csv(dirPath+'gennembrudsinfektioner_table1.csv',delimiter=';')

df2_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_cases.csv',delimiter=';')
df2_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_indlagte.csv',delimiter=';')
df2_D = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_dode.csv',delimiter=';')
df2_R = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_repositive.csv',delimiter=';')
df2_Int = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_intensiv.csv',delimiter=';')
df2_T = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_tests.csv',delimiter=';')

df3 = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_alle.csv',delimiter=';',decimal=",")
df3_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_cases.csv',delimiter=';',decimal=",")
df3_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_indlagte.csv',delimiter=';',decimal=",")
df3_D = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_dode.csv',delimiter=';',decimal=",")
df3_Int = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_intensiv.csv',delimiter=';',decimal=",")
df3_T = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_tests.csv',delimiter=';',decimal=",")

Path to latest Tuesday was SSI_data_2022-02-08


In [13]:
# weekDTs
df3_D.iloc[:,:11]

,Aldersgruppe,Uge 05-2022_Ingen vaccination,Uge 05-2022_Første vaccination,Uge 05-2022_Anden vaccination,Uge 05-2022_Fuld effekt efter primært forløb,Uge 05-2022_Fuld effekt efter revaccination,Uge 04-2022_Ingen vaccination,Uge 04-2022_Første vaccination,Uge 04-2022_Anden vaccination,Uge 04-2022_Fuld effekt efter primært forløb,Uge 04-2022_Fuld effekt efter revaccination
0,0-5,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6-11,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12-15,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16-19,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20-29,0.0,0,0.0,0.0,0.0,0.0,0.0,0.6,0.7,0.0
5,30-39,0.0,0,0.4,0.5,0.3,0.0,0.0,0.2,0.4,0.0
6,40-49,0.0,0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2
7,50-59,5.0,0,0.5,0.0,0.5,2.4,32.8,0.4,2.9,0.2
8,60-64,0.0,0,1.3,13.1,0.7,14.9,0.0,1.0,0.0,1.0
9,65-69,0.0,0,2.1,10.3,1.8,9.9,139.1,3.7,41.6,2.1


In [14]:

weekDTs = [np.datetime64(datetime.datetime.strptime(d[-4:] + '-W'+d[4:6]+'-1', "%Y-W%W-%w")) for d in df1.Ugenummer]

curOrder = np.argsort(weekDTs)
weekDTs = np.array(weekDTs)
weekDTs = weekDTs[curOrder]
        
sNone = 'Ingen vaccination'        
sOne = 'Første vaccination'  
sTwo = 'Anden vaccination'
sFull = 'Fuld effekt efter primært forløb'   
sReva = 'Fuld effekt efter revaccination'


ageGroups = df2_C.Aldersgruppe.values
# print(ageGroups)
# weekNames = df1.Ugenummer
weekNames = df1.Ugenummer.values[curOrder]
weekNamesShort = [x[4:6] for x in weekNames]
wInt = [int(x[4:6]) for x in weekNames]
wIntRange = np.arange(len(wInt))

allDates = np.array(weekDTs)[curOrder]
print(weekNames)

# Make function for gettings particular parts
def getTimeSeriesSingle(thisdf=df2_C,curStatus='Ingen vaccination',curAge='Alle',weekNames=weekNames):
    
    agedf = thisdf[thisdf.Aldersgruppe==curAge]
    allVals = []
    for curWeek in weekNames:
        toAdd = agedf[curWeek+'_'+curStatus].values[0]
        allVals.append(toAdd)
    allVals = np.array(allVals)

    return allVals


def getTimeSeries(thisdf=df2_C,curStatus='Ingen vaccination',curAge='Alle',weekNames=weekNames):
    # If curAge is just a string, return the given array
    if (type(curAge)==str):
        agedf = thisdf[thisdf.Aldersgruppe==curAge]
        allVals = []
        for curWeek in weekNames:
            toAdd = agedf[curWeek+'_'+curStatus].values[0]
            allVals.append(toAdd)
        allVals = np.array(allVals)
        
        return allVals 
        
    else:
        # If curAge is not a string, it is assumed to be a list of strings
        totCount = []
        for thisAge in curAge:
            agedf = thisdf[thisdf.Aldersgruppe==thisAge]
            allVals = []
            for curWeek in weekNames:
                toAdd = agedf[curWeek+'_'+curStatus].values[0]
                allVals.append(toAdd)
            allVals = np.array(allVals)
            if len(totCount) == 0:
                totCount = allVals 
            else:
                totCount = totCount + allVals
        return totCount

def getTimeSeriesAll(thisdf=df2_C,curAge='Alle',weekNames=weekNames):
    return getTimeSeries(thisdf,sNone,curAge,weekNames)+getTimeSeries(thisdf,sOne,curAge,weekNames)+getTimeSeries(thisdf,sTwo,curAge,weekNames)

['Uge 42-2021' 'Uge 43-2021' 'Uge 44-2021' 'Uge 45-2021' 'Uge 46-2021'
 'Uge 47-2021' 'Uge 48-2021' 'Uge 49-2021' 'Uge 50-2021' 'Uge 51-2021'
 'Uge 52-2021' 'Uge 01-2022' 'Uge 02-2022' 'Uge 03-2022' 'Uge 04-2022'
 'Uge 05-2022']


In [15]:
print(ageGroups)

print(getTimeSeriesAll(df2_D,curAge=['70-79','80+']))
print(getTimeSeriesAll(df2_D,curAge='70-79'))
print(getTimeSeriesAll(df2_D,curAge='80+'))

['0-5' '6-11' '12-15' '16-19' '20-29' '30-39' '40-49' '50-59' '60-64'
 '65-69' '70-79' '80+' '12+' 'Alle']
[  8  14  18  25  33  62  60  54  63  73  56  82  91  88 103 120]
[ 3  4  3  8  9 23 18 22 21 27 23 27 25 20 38 26]
[ 5 10 15 17 24 39 42 32 42 46 33 55 66 68 65 94]


# Compare "gennembruds" files with regional files

In [16]:
df_tot = df.groupby('Prøvetagningsdato').sum()
datesDaily = df_tot.index
deathsDaily = df_tot['Døde'].values

In [17]:
fig,ax1 = plt.subplots()

ax2 = ax1.twinx()

ax1.plot(datesDaily,deathsDaily,'k.-',markersize=3,linewidth=0.5,label='Daglig data, regionalt')
ax1.plot(rnTime(datesDaily,7),rnMean(deathsDaily,7),'k',label='7-dages gennemsnit, regionalt')
weekTotDeaths = getTimeSeriesAll(df2_D)
# ax2.step(weekDTs,weekTotDeaths,'b',label='Ugentlige data, gennembrudsfiler')
ax1.step(weekDTs,weekTotDeaths/7,'b',where='pre',label='Ugentlig data, gennembrudsfiler')

ax1.set_xlim(left=weekDTs[0],right=datesDaily[-1]+np.timedelta64(5,'D'))

ax1.set_ylabel('Daglige dødsfald')
ax2.set_ylabel('Ugentlige dødsfald')

ax1.set_ylim(bottom=0)
ax1.set_ylim(top=np.max(deathsDaily[-50:])*1.1)

ax1.legend()

curylim = ax1.get_ylim()
ax2.set_ylim(0,curylim[1]*7)

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

# Draw weekends
firstSunday = np.datetime64('2021-10-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.grid(axis='y')

if saveFigures:
    plt.savefig(path_figs+'Regionalt\\DeathsComparison')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Ugentlige dødsfald, fordelt på alder

In [18]:
ageGroupsGrouped = [ageGroups[0:5],ageGroups[5:7],ageGroups[7:10],ageGroups[10],ageGroups[11]]
ageGroupsGroupedNames = ['0-29','30-49','50-69','70-79','80+']
print(ageGroupsGrouped)
print(ageGroupsGroupedNames)

[array(['0-5', '6-11', '12-15', '16-19', '20-29'], dtype=object), array(['30-39', '40-49'], dtype=object), array(['50-59', '60-64', '65-69'], dtype=object), '70-79', '80+']
['0-29', '30-49', '50-69', '70-79', '80+']


In [19]:
firstDate = weekDTs[0]

In [20]:
fig,ax1 = plt.subplots(figsize=(13,6.5))
fig.patch.set_facecolor('xkcd:off white')
ax1.set_facecolor('xkcd:off white')

# curAge = ageGroups[-3]
# curAge
# for curAge in ageGroupsGrouped:
for i in range(len(ageGroupsGrouped)):
    curAge = ageGroupsGrouped[i]
    curName = ageGroupsGroupedNames[i]

    curDeaths = getTimeSeriesAll(df2_D,curAge=curAge)
    ax1.plot(weekDTs,curDeaths,'.-',label=curName)
    # if (np.sum(curDeaths) > 0):
    #     ax1.plot(weekDTs,curDeaths,label=curAge)

# Draw weekends
firstSunday = np.datetime64('2021-10-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentlige nye dødsfald')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
# ax1.set_xlim(left=firstDate,right=thisdf.Dato.max()+np.timedelta64(5,'D'))
ax1.set_xlim(left=firstDate,right=weekDTs[-1]+np.timedelta64(5,'D'))
ax1.set_ylim(bottom=0)
# ax2.set_ylim(bottom=0)
ax1.legend(loc='upper left')
# ax2.legend(loc='upper left')
ax1.grid(axis='y')
# ax2.grid(axis='y')

# smalltxt = 'Rasmus Kristoffer Pedersen, PandemiX Center, Roskilde Universitet.\nData fra SSI overvågningsdata, dashboard-fil. Filnavn: Regionalt_DB/18_fnkt_alder_uge_testede_positive_nyindlagte.csv'
# ax1.text(1,-0.15,smalltxt,
#      horizontalalignment='right',
#      verticalalignment='top',
#      transform = ax1.transAxes,
#      fontsize=10)
     
fig.tight_layout()


fig.savefig(path_figs+'AgeDistribution\\Agedistribution_DeathsWeekly')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fig,ax1 = plt.subplots(figsize=(13,6.5))
fig.patch.set_facecolor('xkcd:off white')
ax1.set_facecolor('xkcd:off white')

# curAge = ageGroups[-3]
# curAge
# for curAge in ageGroupsGrouped:
for i in range(len(ageGroupsGrouped)):
    curAge = ageGroupsGrouped[i]
    curName = ageGroupsGroupedNames[i]

    curDeaths = getTimeSeriesAll(df3_D,curAge=curAge)
    ax1.plot(weekDTs,curDeaths/1000,'.-',label=curName)
    # if (np.sum(curDeaths) > 0):
    #     ax1.plot(weekDTs,curDeaths,label=curAge)

# Draw weekends
firstSunday = np.datetime64('2021-10-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentlige nye dødsfald per 100 borgere [%]')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
# ax1.set_xlim(left=firstDate,right=thisdf.Dato.max()+np.timedelta64(5,'D'))
ax1.set_xlim(left=firstDate,right=weekDTs[-1]+np.timedelta64(5,'D'))
ax1.set_ylim(bottom=0)
# ax2.set_ylim(bottom=0)
ax1.legend(loc='upper left')
# ax2.legend(loc='upper left')
ax1.grid(axis='y')
# ax2.grid(axis='y')

# smalltxt = 'Rasmus Kristoffer Pedersen, PandemiX Center, Roskilde Universitet.\nData fra SSI overvågningsdata, dashboard-fil. Filnavn: Regionalt_DB/18_fnkt_alder_uge_testede_positive_nyindlagte.csv'
# ax1.text(1,-0.15,smalltxt,
#      horizontalalignment='right',
#      verticalalignment='top',
#      transform = ax1.transAxes,
#      fontsize=10)
     
fig.tight_layout()


fig.savefig(path_figs+'AgeDistribution\\Agedistribution_DeathsWeeklyNormed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
fig,ax1 = plt.subplots(figsize=(13,6.5))
fig.patch.set_facecolor('xkcd:off white')
ax1.set_facecolor('xkcd:off white')

# curAge = ageGroups[-3]
# curAge
# for curAge in ageGroupsGrouped:
for i in range(len(ageGroupsGrouped)):
    curAge = ageGroupsGrouped[i]
    curName = ageGroupsGroupedNames[i]

    curDeaths = getTimeSeriesAll(df3_D,curAge=curAge)
    ax1.plot(weekDTs,np.cumsum(curDeaths/1000),'.-',label=curName)
    # if (np.sum(curDeaths) > 0):
    #     ax1.plot(weekDTs,curDeaths,label=curAge)

# Draw weekends
firstSunday = np.datetime64('2021-10-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylabel('Kumulerede dødsfald per 100 borgere [%]')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
# ax1.set_xlim(left=firstDate,right=thisdf.Dato.max()+np.timedelta64(5,'D'))
ax1.set_xlim(left=firstDate,right=weekDTs[-1]+np.timedelta64(5,'D'))
ax1.set_ylim(bottom=0)
# ax2.set_ylim(bottom=0)
ax1.legend(loc='upper left')
# ax2.legend(loc='upper left')
ax1.grid(axis='y')
# ax2.grid(axis='y')

# smalltxt = 'Rasmus Kristoffer Pedersen, PandemiX Center, Roskilde Universitet.\nData fra SSI overvågningsdata, dashboard-fil. Filnavn: Regionalt_DB/18_fnkt_alder_uge_testede_positive_nyindlagte.csv'
# ax1.text(1,-0.15,smalltxt,
#      horizontalalignment='right',
#      verticalalignment='top',
#      transform = ax1.transAxes,
#      fontsize=10)
     
fig.tight_layout()


fig.savefig(path_figs+'AgeDistribution\\Agedistribution_DeathsWeeklyNormed_Cumulated')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
# fig,ax1 = plt.subplots(figsize=(13,6.5))
# fig.patch.set_facecolor('xkcd:off white')
# ax1.set_facecolor('xkcd:off white')

# # curAge = ageGroups[-3]
# # curAge
# # for curAge in ageGroupsGrouped:
# for i in range(len(ageGroupsGrouped)):
#     curAge = ageGroupsGrouped[i]
#     curName = ageGroupsGroupedNames[i]

#     curDeaths = getTimeSeriesAll(df3_D,curAge=curAge)
#     ax1.plot(weekDTs,curDeaths,'.-',label=curName)
#     # if (np.sum(curDeaths) > 0):
#     #     ax1.plot(weekDTs,curDeaths,label=curAge)

# # Draw weekends
# firstSunday = np.datetime64('2021-10-03')
# numWeeks = 52
# for k in range(0,numWeeks):
#     curSunday = firstSunday + np.timedelta64(7*k,'D')
#     ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

# ax1.set_ylabel('Ugentlige nye dødsfald per 100.000 borgere')

# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
# # ax1.set_xlim(left=firstDate,right=thisdf.Dato.max()+np.timedelta64(5,'D'))
# ax1.set_xlim(left=firstDate,right=weekDTs[-1]+np.timedelta64(5,'D'))
# ax1.set_ylim(bottom=0)
# # ax2.set_ylim(bottom=0)
# ax1.legend(loc='upper left')
# # ax2.legend(loc='upper left')
# ax1.grid(axis='y')
# # ax2.grid(axis='y')

# # smalltxt = 'Rasmus Kristoffer Pedersen, PandemiX Center, Roskilde Universitet.\nData fra SSI overvågningsdata, dashboard-fil. Filnavn: Regionalt_DB/18_fnkt_alder_uge_testede_positive_nyindlagte.csv'
# # ax1.text(1,-0.15,smalltxt,
# #      horizontalalignment='right',
# #      verticalalignment='top',
# #      transform = ax1.transAxes,
# #      fontsize=10)
     
# fig.tight_layout()


# fig.savefig(path_figs+'AgeDistribution\\Agedistribution_DeathsWeeklyNormed')

In [23]:
print(getTimeSeries(df3_D,curStatus=sNone,curAge='80+'))
print(getTimeSeries(df3_D,curStatus=sTwo,curAge='80+'))

[ 32.1  10.8  54.4  33.   55.9 102.1  69.1  81.9  95.6  99.1  76.2  90.5
 132.1  80.5  81.9 303.6]
[ 0.7  2.9  3.7  5.1  6.6 10.7 12.9  9.2 12.5 12.9 10.  17.  20.  23.
 21.6 27. ]


In [37]:
fig,ax1 = plt.subplots(figsize=(13,6.5))
fig.patch.set_facecolor('xkcd:off white')
ax1.set_facecolor('xkcd:off white')

# curAge = ageGroups[-3]
# curAge
# for curAge in ageGroupsGrouped:
for i in range(len(ageGroupsGrouped)):
    curAge = ageGroupsGrouped[i]
    curName = ageGroupsGroupedNames[i]

    # curDeaths = getTimeSeriesAll(df3_D,curAge=curAge)
    curDeaths = getTimeSeries(df3_D,curStatus=sNone,curAge=curAge)
    # ax1.plot(weekDTs,curDeaths,'.-',label=curName)
    ax1.plot(weekDTs,curDeaths/1000,'.-',label=curName)
    # if (np.sum(curDeaths) > 0):
    #     ax1.plot(weekDTs,curDeaths,label=curAge)

ax1.set_title('Ingen vaccination')

# Draw weekends
firstSunday = np.datetime64('2021-10-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentlige nye dødsfald per 100.000 borgere')
ax1.set_ylabel('Ugentlige nye dødsfald per 100 borgere [%]')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
# ax1.set_xlim(left=firstDate,right=thisdf.Dato.max()+np.timedelta64(5,'D'))
ax1.set_xlim(left=firstDate,right=weekDTs[-1]+np.timedelta64(5,'D'))
ax1.set_ylim(bottom=0)
# ax2.set_ylim(bottom=0)
ax1.legend(loc='upper left')
# ax2.legend(loc='upper left')
ax1.grid(axis='y')
# ax2.grid(axis='y')

# smalltxt = 'Rasmus Kristoffer Pedersen, PandemiX Center, Roskilde Universitet.\nData fra SSI overvågningsdata, dashboard-fil. Filnavn: Regionalt_DB/18_fnkt_alder_uge_testede_positive_nyindlagte.csv'
# ax1.text(1,-0.15,smalltxt,
#      horizontalalignment='right',
#      verticalalignment='top',
#      transform = ax1.transAxes,
#      fontsize=10)
 
 
deathUvaccYlim = ax1.get_ylim()     
     
fig.tight_layout()


fig.savefig(path_figs+'AgeDistribution\\Agedistribution_DeathsWeeklyNormed_IngenVaccination')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
fig,ax1 = plt.subplots(figsize=(13,6.5))
fig.patch.set_facecolor('xkcd:off white')
ax1.set_facecolor('xkcd:off white')

# curAge = ageGroups[-3]
# curAge
# for curAge in ageGroupsGrouped:
for i in range(len(ageGroupsGrouped)):
    curAge = ageGroupsGrouped[i]
    curName = ageGroupsGroupedNames[i]

    # curDeaths = getTimeSeriesAll(df3_D,curAge=curAge)
    curDeaths = getTimeSeries(df3_D,curStatus=sTwo,curAge=curAge)
    # ax1.plot(weekDTs,curDeaths,'.-',label=curName)
    ax1.plot(weekDTs,curDeaths/1000,'.-',label=curName)
    # if (np.sum(curDeaths) > 0):
    #     ax1.plot(weekDTs,curDeaths,label=curAge)

ax1.set_title('To eller tre vaccinationer')

# Draw weekends
firstSunday = np.datetime64('2021-10-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentlige nye dødsfald per 100.000 borgere')
ax1.set_ylabel('Ugentlige nye dødsfald per 100 borgere [%]')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
# ax1.set_xlim(left=firstDate,right=thisdf.Dato.max()+np.timedelta64(5,'D'))
ax1.set_xlim(left=firstDate,right=weekDTs[-1]+np.timedelta64(5,'D'))
ax1.set_ylim(bottom=0)
# ax2.set_ylim(bottom=0)
ax1.legend(loc='upper left')
# ax2.legend(loc='upper left')
ax1.grid(axis='y')
# ax2.grid(axis='y')

# smalltxt = 'Rasmus Kristoffer Pedersen, PandemiX Center, Roskilde Universitet.\nData fra SSI overvågningsdata, dashboard-fil. Filnavn: Regionalt_DB/18_fnkt_alder_uge_testede_positive_nyindlagte.csv'
# ax1.text(1,-0.15,smalltxt,
#      horizontalalignment='right',
#      verticalalignment='top',
#      transform = ax1.transAxes,
#      fontsize=10)
     
# ax1.set_ylim(top=deathUvaccYlim[1])
fig.tight_layout()


fig.savefig(path_figs+'AgeDistribution\\Agedistribution_DeathsWeeklyNormed_TwoOrMoreVacc')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Make bar graph of both age and vacc-status

In [69]:

# curDeaths = getTimeSeries(df3_D,curStatus=sNone,curAge=curAge)
# inci_unvacc = curDeaths[-4:].sum()
allInci_unvacc

[0.0, 0.0018, 0.0902, 0.1103, 0.5981000000000001]

In [40]:

curDeaths = getTimeSeries(df2_D,curStatus=sNone,curAge=curAge)

allCount_unvacc = []
allCount_vacc = []
allInci_unvacc = []
allInci_vacc = []
for i in range(len(ageGroupsGrouped)):
    curAge = ageGroupsGrouped[i]
    curName = ageGroupsGroupedNames[i]
    
    curDeaths = getTimeSeries(df2_D,curStatus=sNone,curAge=curAge)
    count_unvacc = curDeaths[-4:].sum()
    curDeaths = getTimeSeries(df2_D,curStatus=sTwo,curAge=curAge)
    count_vacc = curDeaths[-4:].sum()
    
    curDeaths = getTimeSeries(df3_D,curStatus=sNone,curAge=curAge)
    inci_unvacc = curDeaths[-4:].sum()
    curDeaths = getTimeSeries(df3_D,curStatus=sTwo,curAge=curAge)
    inci_vacc = curDeaths[-4:].sum()
    
    allCount_unvacc.append(count_unvacc)
    allCount_vacc.append(count_vacc)
    allInci_unvacc.append(inci_unvacc)
    allInci_vacc.append(inci_vacc)
    
fig,(ax1,ax2) = plt.subplots(2,1)

fig.patch.set_facecolor('xkcd:off white')
for ax in (ax1,ax2):
    ax.set_facecolor('xkcd:off white')
    
curX = np.arange(len(ageGroupsGroupedNames))
ax1.bar(curX-0.2,allCount_unvacc,width=0.4,color='xkcd:purple',label='Ingen vaccination')
ax1.bar(curX+0.2,allCount_vacc,width=0.4,color='xkcd:dark orange',label='To eller tre vaccinationer')
ax1.set_xticks(curX)
ax1.set_xticklabels(ageGroupsGroupedNames)

ax2.bar(curX-0.2,allInci_unvacc,width=0.4,color='xkcd:purple',label='Ingen vaccination')
ax2.bar(curX+0.2,allInci_vacc,width=0.4,color='xkcd:dark orange',label='To eller tre vaccinationer')
ax2.set_xticks(curX)
ax2.set_xticklabels(ageGroupsGroupedNames)

ax1.set_ylabel('Antal dødsfald')
ax2.set_ylabel('Dødsfald per 100.000')
ax1.legend(loc='upper left')
ax2.legend(loc='upper left')

ax1.set_axisbelow(True)
ax2.set_axisbelow(True)
ax1.grid(axis='y')
ax2.grid(axis='y')

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
# 100000*np.array(allCount_unvacc)/np.array(allInci_unvacc)

curAge = '80+'

def estimatePopCount(curStatus,curAge):
    if (type(curAge)==str):
        curCase = getTimeSeries(df2_C,curStatus=curStatus,curAge=curAge)
        curInci = getTimeSeries(df3_C,curStatus=curStatus,curAge=curAge)

        curPopCount = 100000*curCase/curInci
        # print(curPopCount)
        avgPopCount = np.mean(curPopCount[-4:])
        avgPopCount
        
        return avgPopCount
    else:
        totPopCount = 0
        for thisAge in curAge:
            curCase = getTimeSeries(df2_C,curStatus=curStatus,curAge=thisAge)
            curInci = getTimeSeries(df3_C,curStatus=curStatus,curAge=thisAge)

            curPopCount = 100000*curCase/curInci
            avgPopCount = np.mean(curPopCount[-4:])
            
            totPopCount = totPopCount + avgPopCount 
        return totPopCount
        
        
# estimatePopCount(sTwo,'80+')
estimatePopCount(sNone,['50-59','60-64','65-69'])
getTimeSeries(df3_D,curStatus=sNone,curAge=['50-59','60-64','65-69'])[-4:].sum()

90.2

In [95]:
testAge = ['50-59','60-64','65-69']
# testAge = '65-69'
print(getTimeSeries(df3_D,curStatus=sNone,curAge=testAge))
print(getTimeSeries(df3_D,curStatus=sNone,curAge=testAge)[-4:].sum())
print(getTimeSeries(df2_D,curStatus=sNone,curAge=testAge))
print(getTimeSeries(df2_D,curStatus=sNone,curAge=testAge)[-4:].sum())

print(estimatePopCount(sNone,testAge))

100000*44/7400
100000*9/41400
100000*4/13552
100000*4/10120
(100000*9/41400) +(100000*4/13552) +(100000*4/10120) 
(41400) +(13552) +(10120) 
# 100000*(4+4+9)/(41400+13552+10120)

[ 0.   1.9  0.   8.3  3.9 14.9 17.3 11.  33.7 11.2  9.3 45.4 23.9 34.1
 27.2  5. ]
90.2
[0 1 0 1 2 2 3 2 5 3 2 5 5 6 4 2]
17
65073.08896805468


65072

In [96]:
# Same figure as above, but with labels with population counts

curDeaths = getTimeSeries(df2_D,curStatus=sNone,curAge=curAge)

allCount_unvacc = []
allCount_vacc = []
allInci_unvacc = []
allInci_vacc = []

popCounts_unvacc = []
popCounts_vacc = []
for i in range(len(ageGroupsGrouped)):
    curAge = ageGroupsGrouped[i]
    curName = ageGroupsGroupedNames[i]
    
    curDeaths = getTimeSeries(df2_D,curStatus=sNone,curAge=curAge)
    count_unvacc = curDeaths[-4:].sum()
    curDeaths = getTimeSeries(df2_D,curStatus=sTwo,curAge=curAge)
    count_vacc = curDeaths[-4:].sum()
    
    curDeaths = getTimeSeries(df3_D,curStatus=sNone,curAge=curAge)
    inci_unvacc = curDeaths[-4:].sum()
    curDeaths = getTimeSeries(df3_D,curStatus=sTwo,curAge=curAge)
    inci_vacc = curDeaths[-4:].sum()
    
    # Scale to % instead of per 100.000
    inci_unvacc = inci_unvacc/1000
    inci_vacc = inci_vacc/1000
    
    allCount_unvacc.append(count_unvacc)
    allCount_vacc.append(count_vacc)
    allInci_unvacc.append(inci_unvacc)
    allInci_vacc.append(inci_vacc)
    
    popCounts_unvacc.append(estimatePopCount(sNone,curAge))
    popCounts_vacc.append(estimatePopCount(sTwo,curAge))
    
fig,(ax1,ax2) = plt.subplots(2,1)

fig.patch.set_facecolor('xkcd:off white')
for ax in (ax1,ax2):
    ax.set_facecolor('xkcd:off white')

dX = 0.2
    
curX = np.arange(len(ageGroupsGroupedNames))
ax1.bar(curX-dX,allCount_unvacc,width=2*dX,color='xkcd:purple',label='Ingen vaccination')
ax1.bar(curX+dX,allCount_vacc,width=2*dX,color='xkcd:dark orange',label='To eller tre vaccinationer')
ax1.set_xticks(curX)
ax1.set_xticklabels(ageGroupsGroupedNames)

ax2.bar(curX-dX,allInci_unvacc,width=2*dX,color='xkcd:purple',label='Ingen vaccination')
ax2.bar(curX+dX,allInci_vacc,width=2*dX,color='xkcd:dark orange',label='To eller tre vaccinationer')
ax2.set_xticks(curX)
ax2.set_xticklabels(ageGroupsGroupedNames)


# Round popcounts to nearest hundred
popCounts_unvacc = [int(np.round(x/100))*100 for x in popCounts_unvacc]
popCounts_vacc = [int(np.round(x/100))*100 for x in popCounts_vacc]
# popCounts_unvacc = [int(np.round(x/1000))*1000 for x in popCounts_unvacc]
# popCounts_vacc = [int(np.round(x/1000))*1000 for x in popCounts_vacc]
# popCounts_unvacc = [int(np.round(x/1000)) for x in popCounts_unvacc]
# popCounts_vacc = [int(np.round(x/1000)) for x in popCounts_vacc]
# Add counts to figure
for i in curX:
    ax1.text(i-dX,allCount_unvacc[i]+2,allCount_unvacc[i],ha='center')
    ax1.text(i+dX,allCount_vacc[i]+2,allCount_vacc[i],ha='center')
    
    curStr = f'{allCount_unvacc[i]} ud af\n{popCounts_unvacc[i]:n}'
    ax2.text(i-dX,allInci_unvacc[i]+10,curStr,ha='center')
    curStr = f'{allCount_vacc[i]} ud af\n{popCounts_vacc[i]:n}'
    ax2.text(i+dX,allInci_vacc[i]+10,curStr,ha='center')
    # curStr = f'{allCount_unvacc[i]} ud af\n{popCounts_unvacc[i]} tusind'
    # ax2.text(i-dX,allInci_unvacc[i]+10,curStr,ha='center')
    # curStr = f'{allCount_vacc[i]} ud af\n{popCounts_vacc[i]} tusind'
    # ax2.text(i+dX,allInci_vacc[i]+10,curStr,ha='center')

ax2.set_ylim(top=allInci_unvacc[-1]*1.2)

ax1.set_ylabel('Antal dødsfald')
# ax2.set_ylabel('Dødsfald per 100.000')
ax2.set_ylabel('Andel af befolkning [%]')
ax1.legend(loc='upper left')
ax2.legend(loc='upper left')

ax1.set_axisbelow(True)
ax2.set_axisbelow(True)
ax1.grid(axis='y')
ax2.grid(axis='y')

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

strTitle = f'Danske COVID-19 registrerede dødsfald\nPeriode fra og med uge {weekNamesShort[-4]} til og med uge {weekNamesShort[-1]}'
fig.suptitle(strTitle,fontsize=20)

txt1 = 'Kategorien "Første vaccination" er udeladt grundet små tal. Bemærk: Nøjagtige befolkningstal fordelt på vaccinationsstatus er ikke tilgængelige,\nog er her estimeret ud fra 4-ugers gennemsnit af antal og incidens per 100.000 og derefter afrundet til nærmeste hundrede.\nRasmus Kristoffer Pedersen, PandemiX Center, Roskilde Universitet. Data fra SSI overvågningsdata, gennembrudsdatafiler.'
ax2.text(1, -0.14,txt1,
     horizontalalignment='right',
     verticalalignment='top',
     transform = ax2.transAxes,
     fontsize=7)
     
fig.tight_layout()

if saveFigures:
    fig.savefig(path_figs+'AgeDistribution\\DoesdfaldPerVaccinestatusOgAlder')

<ipython-input-92-7d52d2fc18b2>:22: RuntimeWarning: invalid value encountered in true_divide
  curPopCount = 100000*curCase/curInci


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-96-c8a702d3753f>:107: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  fig.tight_layout()


In [97]:
allInci_unvacc * np.array(popCounts_vacc) / 100000
diffInci = np.array(allInci_unvacc)/np.array(allInci_vacc)
deathsEstiTot = diffInci * np.array(allCount_vacc)
deathsEstiTotRound = np.array([int(np.round(x)) for x in deathsEstiTot])
deathsDiffRound = deathsEstiTotRound - allCount_vacc
# deathsAvoid = deathsEstiTot -allCount_vacc
# deathsDiffRound = np.array([int(np.round(x)) for x in deathsAvoid])

In [98]:
deathsEstiTot
deathsEstiTotRound
deathsDiffRound

array([  -4,    1,  263,  503, 1360], dtype=int64)

In [99]:
# Comparison of deaths in vacc and unvacc groups, and estimate of number of deaths avoided

fig,ax1 = plt.subplots(figsize=(13,6))

fig.patch.set_facecolor('xkcd:off white')
ax1.set_facecolor('xkcd:off white')

dX = 0.2
    
ax1.bar(curX-dX,allInci_unvacc,width=2*dX,color='xkcd:purple',label='Ingen vaccination')
ax1.bar(curX+dX,allInci_vacc,width=2*dX,color='xkcd:dark orange',label='To eller tre vaccinationer',zorder=3)
ax1.bar(curX+dX,allInci_unvacc,width=2*dX,color='xkcd:light grey',label='Undgåede dødsfald',zorder=2,hatch='\\\\',edgecolor='xkcd:orange')
ax1.set_xticks(curX)
ax1.set_xticklabels(ageGroupsGroupedNames)


bboxStyle = dict(facecolor='white',alpha=0.75, edgecolor='grey', boxstyle='round')
curOffset = 5/1000
# Add counts to figure
# for i in curX:
for i in curX[2:]:
    # ax1.text(i-dX,allCount_unvacc[i]+2,allCount_unvacc[i],ha='center')
    # ax1.text(i+dX,allCount_vacc[i]+2,allCount_vacc[i],ha='center')
    
    curStr = f'{allCount_unvacc[i]} ud af\n{popCounts_unvacc[i]:n}'
    ax1.text(i-dX,allInci_unvacc[i]+curOffset,curStr,ha='center',va='bottom',bbox=bboxStyle)
    curStr = f'Antal dødsfald:{allCount_unvacc[i]}'
    ax1.text(i-dX,allInci_unvacc[i]-curOffset,curStr,ha='center',va='top',bbox=bboxStyle)
    
    curStr = f'Tilsvarende incidens:\n{np.round(deathsEstiTotRound[i]):.0f} ud af\n{popCounts_vacc[i]:n}'
    ax1.text(i+dX,allInci_unvacc[i]+curOffset,curStr,ha='center',va='bottom',bbox=bboxStyle)
    
    curStr = f'Antal dødsfald: {allCount_vacc[i]}'
    ax1.text(i+dX,allInci_vacc[i]-curOffset,curStr,ha='center',va='top',bbox=bboxStyle)
    curStr = f'Undgåede dødsfald:\n{deathsDiffRound[i]}'
    ax1.text(i+dX,allInci_unvacc[i]-curOffset,curStr,ha='center',va='top',bbox=bboxStyle)
    # curStr = f'{allCount_unvacc[i]} ud af\n{popCounts_unvacc[i]} tusind'
    # ax2.text(i-dX,allInci_unvacc[i]+10,curStr,ha='center')
    # curStr = f'{allCount_vacc[i]} ud af\n{popCounts_vacc[i]} tusind'
    # ax2.text(i+dX,allInci_vacc[i]+10,curStr,ha='center')

# ax2.set_ylim(top=allInci_unvacc[-1]*1.2)

ax1.set_ylabel('Dødsfald per 100.000')
ax1.set_ylabel('Dødsfald, andel af befolkning [%]')
ax1.legend(loc='upper left',fontsize=10)

ax1.set_axisbelow(True)
ax1.grid(axis='y')

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

strTitle = f'Danske COVID-19 registrerede dødsfald.\nPeriode fra og med uge {weekNamesShort[-4]} til og med uge {weekNamesShort[-1]}'
ax1.set_title(strTitle,fontsize=14)

# txt1 = 'Kategorien "Første vaccination" er udeladt grundet små tal. Bemærk: Nøjagtige befolkningstal fordelt på vaccinationsstatus er ikke tilgængelige,\nog er her estimeret ud fra 4-ugers gennemsnit af antal og incidens per 100.000 og derefter afrundet til nærmeste hundrede.\nRasmus Kristoffer Pedersen, PandemiX Center, Roskilde Universitet. Data fra SSI overvågningsdata, gennembrudsdatafiler.'
# ax2.text(1, -0.14,txt1,
#      horizontalalignment='right',
#      verticalalignment='top',
#      transform = ax2.transAxes,
#      fontsize=7)

ax1.set_xlim([1.5,4.5])
# ax1.set_ylim([0,690])
ax1.set_ylim([0,690/1000])
if saveFigures:
    fig.savefig(path_figs+'AgeDistribution\\Doesdfaldundgaaet_50+')

ax1.set_xlim([3.5,4.5])

if saveFigures:
    fig.savefig(path_figs+'AgeDistribution\\Doesdfaldundgaaet_80+')
    
ax1.set_xlim([1.5,3.5])
# ax1.set_ylim([0,150])
ax1.set_ylim([0,150/1000])
     
fig.tight_layout()

if saveFigures:
    fig.savefig(path_figs+'AgeDistribution\\Doesdfaldundgaaet_50_79')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-99-4e7733a2adc6>:79: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 
  fig.tight_layout()


# Make some comparison plots

In [32]:
def addWeekends(ax):
    # Draw weekends
    firstSunday = np.datetime64('2021-10-03')
    numWeeks = 52
    for k in range(0,numWeeks):
        curSunday = firstSunday + np.timedelta64(7*k,'D')
        ax.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')


In [33]:


fig,allAxes = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('xkcd:off white')
for ax in allAxes.flatten():
    ax.set_facecolor('xkcd:off white')
# ax1.set_facecolor('xkcd:off white')

i = -1
curAge = ageGroupsGrouped[i]
curName = ageGroupsGroupedNames[i]

ax = allAxes.flatten()[0]
curDeaths = getTimeSeries(df2_D,curStatus=sNone,curAge=curAge)
ax.plot(weekDTs,curDeaths,'.-',label='Ingen vaccination')
curDeaths = getTimeSeries(df2_D,curStatus=sTwo,curAge=curAge)
ax.plot(weekDTs,curDeaths,'.-',label='To eller tre vaccinationer')

addWeekends(ax)
ax.set_ylim(bottom=0)
ax.grid(axis='y')
ax.legend(loc='upper left')
ax.set_ylabel('Antal ugentlige dødsfald')

ax = allAxes.flatten()[1]
curDeaths = getTimeSeries(df3_D,curStatus=sNone,curAge=curAge)
ax.plot(weekDTs,curDeaths,'.-',label='Ingen vaccination')
curDeaths = getTimeSeries(df3_D,curStatus=sTwo,curAge=curAge)
ax.plot(weekDTs,curDeaths,'.-',label='To eller tre vaccinationer')
addWeekends(ax)

ax.set_ylim(bottom=0)
ax.grid(axis='y')
ax.legend(loc='upper left')
ax.set_ylabel('Ugentlige dødsfald\nper 100.000 borgere')

ax.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax.set_xlim(left=firstDate,right=weekDTs[-1]+np.timedelta64(5,'D'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(18918.0, 19028.0)